In [1]:
import pandas as pd
import os


# Construct the path to the data file
data_path = os.path.join('archive', 'stocks', 'A.csv')

# Load the dataset
df = pd.read_csv(data_path)

# Display the first few rows
df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200


In [2]:
# Check for missing values
df.isnull().sum()

# Drop rows with missing values
df.dropna(inplace=True)

# Ensure correct data types
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [3]:
# Example: Create a 7-day moving average
df['MA7'] = df['Close'].rolling(window=7).mean()
# Example: Daily returns
df['Daily_Return'] = df['Close'].pct_change()


In [4]:
import numpy as np

# Feature columns and target
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'Daily_Return']
target = 'Close'

# Create sequences (e.g., using past 60 days to predict the next day)
sequence_length = 60

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data.iloc[i:i + seq_length][features].values)
        labels.append(data.iloc[i + seq_length][target])
    return np.array(sequences), np.array(labels)

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Create sequences
X_train, y_train = create_sequences(train_df, sequence_length)
X_test, y_test = create_sequences(test_df, sequence_length)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, len(features))),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


c:\Users\Usama Mohiuddin\Desktop\Stock_market\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 58, 64)         │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 27, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 832)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │        41,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,461 (216.64 KB)

 Trainable params: 55,461 (216.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 57214046208.0000 - val_loss: 4333.7974
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 843.3581 - val_loss: 4333.7915
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 819.8647 - val_loss: 4333.7847
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 817.4930 - val_loss: 4333.7764
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 843.1040 - val_loss: 4333.7661
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 830.8948 - val_loss: 4333.7544
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 803.3485 - val_loss: 4333.7422
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 832.2167 - val_loss: 4333.7275
Epoch 9/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 826.6486 - val_loss: 4333.7114
Epoch 10/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 820.1364 - val_loss: 4333.6938


In [9]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Set TensorFlow to disable XLA compilation (optional)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

# Define your features and sequence length
features = ['Open', 'High', 'Low', 'Close', 'Volume']
sequence_length = 60

# Construct the path to the data file
data_path = os.path.join('archive', 'stocks', 'A.csv')

def fetch_data_from_csv():
    # Load the dataset
    df = pd.read_csv(data_path)
    # Select the relevant columns
    df = df[features]
    # Ensure the DataFrame has at least 'sequence_length' rows
    if len(df) < sequence_length:
        raise ValueError(f"Dataset must have at least {sequence_length} rows")
    # Use the latest 'sequence_length' rows
    df = df.tail(sequence_length).reset_index(drop=True)
    return df

def preprocess(data):
    try:
        # Normalize the data
        data = (data - data.mean()) / data.std()
        # Reshape the data for the model
        reshaped_data = np.array(data).reshape(1, sequence_length, len(features))
        print(f"Input shape: {reshaped_data.shape}")  # Add this line to print the input shape
        return reshaped_data
    except Exception as e:
        print(f"Preprocessing error: {e}")
        raise

def predict_and_plot():
    try:
        # Fetch data from the CSV file
        data = fetch_data_from_csv()
        processed_data = preprocess(data)
        prediction = model.predict(processed_data)
        
        # Create a plot
        fig, ax = plt.subplots()
        ax.plot(data['Close'], label='Actual Stock Prices')
        ax.set_xlabel('Time')
        ax.set_ylabel('Price')
        ax.set_title('Stock Prices Over Time')
        ax.legend()
        
        # Return the plot and the prediction result
        return fig, f"Predicted Stock Price: {prediction[0][0]}"
    except Exception as e:
        print(f"Prediction error: {e}")
        return None, f"Error: {e}"

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, len(features))),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Gradio interface with plot
iface = gr.Interface(
    fn=predict_and_plot,
    inputs=None,
    outputs=[gr.Plot(), "text"],
    live=True
)

iface.launch()

c:\Users\Usama Mohiuddin\Desktop\Stock_market\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 58, 64)         │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 27, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 832)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │        41,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,077 (215.14 KB)

 Trainable params: 55,077 (215.14 KB)

 Non-trainable params: 0 (0.00 B)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Input shape: (1, 60, 5)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


In [7]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Assuming X_test and y_test are defined and preprocessed correctly
# Assuming model is already trained

# Predict on the test set
predictions = model.predict(X_test)

# Ensure the predictions and y_test have compatible shapes
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of predictions: {predictions.shape}")

# Ensure there are no NaN values
print(f"NaN in y_test: {np.isnan(y_test).sum()}")
print(f"NaN in predictions: {np.isnan(predictions).sum()}")

# If predictions are 2D, convert to 1D
if predictions.ndim > 1:
    predictions = predictions.flatten()

# Calculate the mean squared error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")



Input shape: (1, 60, 5)
Prediction error: Exception encountered when calling Sequential.call().

Input 0 of layer "conv1d" is incompatible with the layer: expected axis -1 of input shape to have value 7, but received input with shape (1, 60, 5)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 60, 5), dtype=float32)
  • training=False
  • mask=None
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Shape of y_test: (965,)
Shape of predictions: (965, 1)
NaN in y_test: 0
NaN in predictions: 0
Mean Squared Error: 4333.694078864977


Input shape: (1, 60, 5)
Prediction error: Graph execution error:

Detected at node convolution defined at (most recent call last):
<stack traces unavailable>
Depth of input must be a multiple of depth of filter: 5 vs 7

Stack trace for op definition: 
File "C:\Users\Usama Mohiuddin\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 995, in _bootstrap
File "C:\Users\Usama Mohiuddin\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
File "c:\Users\Usama Mohiuddin\Desktop\Stock_market\myenv\Lib\site-packages\anyio\_backends\_asyncio.py", line 859, in run
File "c:\Users\Usama Mohiuddin\Desktop\Stock_market\myenv\Lib\site-packages\gradio\utils.py", line 816, in wrapper
File "C:\Users\Usama Mohiuddin\AppData\Local\Temp\ipykernel_13152\2737601351.py", line 46, in predict_and_plot
File "c:\Users\Usama Mohiuddin\Desktop\Stock_market\myenv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
File "c:\Users\Usama Moh